In [10]:
import librosa as lb
import librosa.display
import librosa.feature
import librosa.util
import librosa.effects

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import wave 
import soundfile as sf
import pyaudio

import os
import time

import torch
import torch.nn as nn


In [16]:
def play_wave(wave, wave_info):
    # initialize PyAudio
    p = pyaudio.PyAudio()

    # open a stream
    stream = p.open(format=pyaudio.paFloat32,
                    channels=wave_info.channels,
                    rate=wave_info.samplerate,
                    output=True)

    # play audio
    stream.write(wave.tobytes())
    time.sleep(1)


    # stop stream and terminate PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

In [39]:
dir_path = r"C:\Users\llama\Desktop\cuni\bakalarka\data\test\kicks"
# dir_path = r"C:\Users\llama\Desktop\cuni\bakalarka\data\test\crashes" 

file_paths = [os.path.join(dir_path, path) for path in os.listdir(dir_path)]

In [40]:
class Wave():
    def __init__(self, array, sr, info = None) -> None:
        self.array = array
        self.sr = sr
        self.info = info

waves = []

for path in file_paths:
    array, sr = lb.load(path)
    info = sf.info(path)
    waves.append(Wave(array, sr, info))

In [51]:
def pad_or_trim(mfcc, length = 100):
    if mfcc.shape[1] > length:
        return mfcc[:, :length]
    else:
        last_column = mfcc[:, -1:]
        padding = np.repeat(last_column, length - mfcc.shape[1], axis=1)
        return np.concatenate((mfcc, padding), axis=1)
        # return np.pad(mfcc, ((0,0),(0,length-mfcc.shape[1])), constant_values = mfcc[:,-1])        def pad_or_trim(mfcc, length=100):


In [52]:
mfccs = []

for wave in waves:
    mfcc = lb.feature.mfcc(y=wave.array, sr=wave.sr, n_mfcc=256, n_fft=2048, hop_length=256, lifter=0, dct_type=3, n_mels = 256)
  
    print(f"OG wave duration: {wave.info.duration}")
    print(f"{mfcc[-1][-1]}")
    print(f"OG mfcc shape: {mfcc.shape}")
    mfcc_pad_or_trim = pad_or_trim(mfcc, 100)
    print(f"New mfcc shape: {mfcc_pad_or_trim.shape}")
    print(f"{mfcc_pad_or_trim[-1][-1]}")


    mfccs.append(mfcc_pad_or_trim)

    inverse_pad_or_trim = lb.feature.inverse.mfcc_to_audio(mfcc_pad_or_trim, sr=wave.sr, n_fft=2048, hop_length=256, lifter=0, dct_type=3, n_mels = 256)

    print(f"New wave duration: {inverse_pad_or_trim.shape[0]/wave.sr}")
    play_wave(wave.array, wave.info)
    play_wave(inverse_pad_or_trim, wave.info)




OG wave duration: 0.4342403628117914
-0.18709154427051544
OG mfcc shape: (256, 38)
New mfcc shape: (256, 100)
-0.18709154427051544
New wave duration: 1.1493877551020408
OG wave duration: 0.48387755102040814
0.2104833871126175
OG mfcc shape: (256, 42)
New mfcc shape: (256, 100)
0.2104833871126175
New wave duration: 1.1493877551020408
OG wave duration: 0.5241950113378685
-0.5781711339950562
OG mfcc shape: (256, 46)
New mfcc shape: (256, 100)
-0.5781711339950562
New wave duration: 1.1493877551020408


KeyboardInterrupt: 